In [14]:
## Welcome to Week 5 ##
# Today we are going to try some validation

##Imports##
__author__ = 'bdyetton'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss

In [5]:
#Load and prepare data
all_charts = pd.read_csv('BillboardLyricData.txt', sep='\t', encoding='utf-8')
all_charts = all_charts.dropna() ## Remove missing data

class_mapping = {label:idx for idx,label in enumerate(np.unique(all_charts.chart))}
y_raw = all_charts.chart.map(class_mapping)



In [56]:
vectorizer = CountVectorizer(max_df=0.95, min_df=2,max_features=5000,stop_words='english')                              
vectorizer.fit(all_charts.lyrics) #Creates the dictionary to convert lyrics to counts
word_vector = vectorizer.transform(all_charts.lyrics) #Does actual conversion
X_raw = word_vector.todense()

#Split of test set, and set aside (dont touch until after we have found the best hyperparameters)
X_train_and_valid, X_test, y_train_and_valid, y_test = train_test_split(X_raw, y_raw, test_size=0.3)

#Split of validation and train set from the remaining
X_train, X_valid, y_train, y_valid = train_test_split(X_train_and_valid, y_train_and_valid, test_size=0.3)
    

In [57]:
def bic_score(y_actual, y_proba, k, n):
    LL = log_loss(y_actual,y_p)
    LL = -LL #Because sklearn negates loss by convention
    return -2*LL + k*np.log(n)

def aic_score(y_actual, y_proba, k):
    LL = log_loss(y_actual,y_p)
    LL = -LL #Because sklearn negates loss by convention
    return -2*LL + k*2

In [58]:
model = SGDClassifier(loss='log', n_iter=100, penalty='none')

for n_features in [5,10,100,500,1000,5000]:
    #X is and ordered list of the [normalized] counts of most frequent words. We can slice this 1000 coloumn matrix of the top 1000 wor
    # This will give a training data that is now the top 100 words
    X_train_temp = X_train[:,0:n_features]
    X_valid_temp = X_valid[:,0:n_features]
    model.fit(X_train_temp, y_train) 
    y_p = model.predict_proba(X_train_temp)
    print("BIC:", bic_score(y_train,y_p,X_train_temp.shape[1],X_train_temp.shape[0]))
    print("AIC:", aic_score(y_train,y_p,X_train_temp.shape[1]))
    print("Test Acc: ",model.score(X_valid_temp,y_valid))

# #Use 500 features
# model_2 = SGDClassifier(loss='log', n_iter=100, penalty='none')
# X_train_temp = X_train[:,0:500]
# X_valid_temp = X_valid[:,0:500]
# model_2.fit(X_train_temp, y_train) 
# y_p = model_2.predict_proba(X_valid_temp)
# print("model 2 bic:", bic_score(y_valid,y_p,X_valid_temp.shape[1],X_valid_temp.shape[0]))
# print("model 2 aic:", aic_score(y_valid,y_p,X_valid_temp.shape[1]))
# print(model_2.score(X_valid_temp,y_valid))

# #Use 500 features
# model_2 = SGDClassifier(loss='log', n_iter=100, penalty='none')
# X_train_temp = X_train[:,0:1000]
# X_valid_temp = X_valid[:,0:1000]
# model_2.fit(X_train_temp, y_train) 
# y_p = model_2.predict_proba(X_valid_temp)
# print("model 2 bic:", bic_score(y_valid,y_p,X_valid_temp.shape[1],X_valid_temp.shape[0]))
# print("model 2 aic:", aic_score(y_valid,y_p,X_valid_temp.shape[1]))
# print(model_2.score(X_valid_temp,y_valid))

# #Use all 1000 features
# model_3 = SGDClassifier(loss='log', n_iter=100, penalty='none')
# X_train_temp = X_train
# X_valid_temp = X_valid
# model_3.fit(X_train_temp, y_train) 
# y_p = model_3.predict_proba(X_valid_temp)
# print("model 3 bic:", bic_score(y_valid,y_p,X_valid_temp.shape[1],X_valid_temp.shape[0]))
# print("model 3 aic:", aic_score(y_valid,y_p,X_valid_temp.shape[1]))
# print(model_3.score(X_valid_temp,y_valid))

BIC: 37.3791209744
AIC: 13.6706175011
Test Acc:  0.195054945055
BIC: 71.0508922842
AIC: 23.6338853376
Test Acc:  0.197802197802


C:\Users\bdyet\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


BIC: 677.396197228
AIC: 203.226127763
Test Acc:  0.247252747253
BIC: 3372.56477746
AIC: 1001.71443014
Test Acc:  0.299450549451
BIC: 6742.50375501
AIC: 2000.80306036
Test Acc:  0.315934065934
BIC: 30857.4013612
AIC: 9154.63728173
Test Acc:  0.392857142857
